In [10]:
import torch
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.utils.data import DataLoader
from custom_dataset import CustomDataset

In [11]:
# Define image transformations
transform = Compose([
    Resize((800, 800)),  # Resize the input image to match the model's input shape
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset and dataloader
dataset = CustomDataset("./data/photo", "./data/label", transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0, collate_fn=lambda x: tuple(zip(*x)))


In [12]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from transformers import AutoFeatureExtractor, AutoModelForObjectDetection

extractor = AutoFeatureExtractor.from_pretrained("facebook/detr-resnet-101")
model = AutoModelForObjectDetection.from_pretrained("facebook/detr-resnet-101")

num_classes = 2  # Background + "test_strip"
model.class_labels_classifier = torch.nn.Linear(in_features=model.class_labels_classifier.in_features, out_features=num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = StepLR(optimizer, step_size=1, gamma=0.9)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (images, targets_tuple) in enumerate(dataloader):
        images = torch.stack(images).to(device)
        targets = [{k: v[i].to(device) for k, v in zip(targets_tuple._fields, targets_tuple)} for i in range(len(targets_tuple.target))]
        optimizer.zero_grad()
        outputs = model(images, targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")





AttributeError: 'tuple' object has no attribute 'target'